In [9]:
import nltk
import pandas as pd
from textblob import Word
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 

data = pd.read_csv('train_data.csv')

# Pre-Processing the text 
def cleaning(df, stop_words):
    df['Text'] = df['Text'].astype(str)  # Convert to string
    df['Text'] = df['Text'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    # Replacing the digits/numbers
    df['Text'] = df['Text'].str.replace(r'\d+', '', regex=True)
    # Removing stop words
    df['Text'] = df['Text'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words))
    # Lemmatization
    df['Text'] = df['Text'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))
    return df

nltk.download('stopwords')
stop_words = stopwords.words('english')
data = data.dropna(subset=['Text'])
data_cleaned = cleaning(data, stop_words)

tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(data_cleaned['Text'].values)
X = tokenizer.texts_to_sequences(data_cleaned['Text'].values)
X = pad_sequences(X)

# Encoding the target variable
Y = pd.get_dummies(data_cleaned['Sentiment']).values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#Trainig
model = Sequential()
model.add(Embedding(500, 120, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(704, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(352, activation='relu'))  # Changed LeakyReLU to relu
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())
model.fit(X_train, y_train, epochs=30, batch_size=128, verbose=1)

#Testing
score = model.evaluate(X_test, y_test)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rudospiew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 21, 120)           60000     
                                                                 
 spatial_dropout1d_1 (Spati  (None, 21, 120)           0         
 alDropout1D)                                                    
                                                                 
 lstm_1 (LSTM)               (None, 704)               2323200   
                                                                 
 dense_2 (Dense)             (None, 352)               248160    
                                                                 
 dense_3 (Dense)             (None, 3)                 1059      
                                                                 
Total params: 2632419 (10.04 MB)
Trainable params: 2632419 (10.04 MB)
Non-trainable params: 0 (0.00 Byte)
______________

In [10]:
model.save('model_big.h5')

/home/rudospiew/.local/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: my_model_bigkeras/assets


INFO:tensorflow:Assets written to: my_model_bigkeras/assets
